# McDollibee - Expansion Recommendation

## Executive Summary

In this notebook we will be sharing the details on how we built the strategy for the **Branch Expansion of McDollibee** by utilizing the SQL PostGIS functions and Python. We have approached this by applying the requirements from the problem statement utilizing the the gadm.ph and public.ph_point data set. This has helped us process in reducing processing time and easily narrow down to the identified locations. Please note that the analysis does not include the population count within the area and something we can consider if we want to extend the solution further. We can look at the results at different angles of the data and do further analysis on each of the shortlisted locations.



## Problem Statement

Suppose that you are the *head of branch expansion* of food chain "McDollibee" with the base of operations located here at AIM. You are tasked to create a strategy that considers the following:
1. Level of Urbanization - You want to expand to a location where the amenity density is in top 10 percentile. 
2. Market Availability and Competition - You cater to students but you prefer areas with limited competition
3. Logistics - The farther the location is from AIM, the more expensive building the branch will be.

Using the information available to you, create a report on what possible locations fit the requirements of your branch expansion strategy.

In [1]:
import psycopg2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-usdluc3_ because the default path (/home/rbanquerigo/.cache/matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


In [2]:
conn = psycopg2.connect(dbname="postgis", 
                 user="gsa2022", 
                 password="g5!V%T1Vmd", 
                 host="192.168.212.99", 
                 port=32771)

### Fetching Schools and University within Metro Manila

We first fetched all schools and universities within Metro Manila as we don't want to go outside given the logistics constraints we have. This would also helps us in reducing the processing time in our query at the later part of the notebook as we have filetered out all other schools/universities within the country.

In [3]:
print('Fetching Schools')
sql = """
          SELECT * FROM gadm.ph g
          INNER JOIN public.ph_point as p
          ON 
          ST_Within(st_transform(p.way, 3123), st_transform(g.geom, 3123))
          WHERE g.name_1 = 'Metropolitan Manila' AND
          (p.amenity = 'school' or p.amenity = 'university')
      """
df_school_mm = pd.read_sql_query(sql, conn)
df_school_mm.head(5)

Fetching Schools


,gid,gid_0,name_0,gid_1,name_1,nl_name_1,gid_2,name_2,varname_2,nl_name_2,...,tourism,tower:type,tunnel,water,waterway,wetland,width,wood,z_order,way
0,969,PHL,Philippines,PHL.47_1,Metropolitan Manila,None,PHL.47.9_1,Navotas,None,None,...,None,None,None,None,None,None,None,None,None,0101000020E6100000E5756968A83B5E4026A36F777B59...
1,969,PHL,Philippines,PHL.47_1,Metropolitan Manila,None,PHL.47.9_1,Navotas,None,None,...,None,None,None,None,None,None,None,None,None,0101000020E61000007FFB9529C13B5E4083E4F8FC8B5B...
2,962,PHL,Philippines,PHL.47_1,Metropolitan Manila,None,PHL.47.2_1,Las Piñas,"Las Piñas, City of",None,...,None,None,None,None,None,None,None,None,None,0101000020E6100000B7ED7BD45F405E404FFD288F13D3...
3,962,PHL,Philippines,PHL.47_1,Metropolitan Manila,None,PHL.47.2_1,Las Piñas,"Las Piñas, City of",None,...,None,None,None,None,None,None,None,None,None,0101000020E61000005094DFFDCC405E406506E055C5D2...
4,968,PHL,Philippines,PHL.47_1,Metropolitan Manila,None,PHL.47.8_1,Muntinlupa,"Muntinlupa, City of",None,...,None,None,None,None,None,None,None,None,None,0101000020E610000059F15712EA415E401FB86F5A95B9...


### Calculate the Density of Schools and Universities

In the query we have calculated the density of amenities within 500 units of the school for us to know how many establishments are within the area. We also took the count of fast food amenities separately as this is the main competitor for McDollibee and this will be one of the key drivers for making the decision on our expansion. 

I can actually add the query for gadm to filter Metro Manila but its taking so long to run and test and best to refactor the code this way for purpose of easier debugging and implemention

**Note: I used 500 as a default since there's no given but its still up to business if they want the distance to be nearer or further**

In [4]:
df_combine = pd.read_sql_query("""
SELECT *, (all_amenity - no_fast_food) as fast_food FROM 
(
    SELECT p1.name as all_name, p1.way as all_way, count(*) as all_amenity
    FROM public.ph_point as p1 
    INNER JOIN public.ph_point as p2
    ON ST_dwithin(st_transform(p1.way,3123), st_transform(p2.way,3123),500)
    WHERE (p1.amenity = 'school' or p1.amenity = 'university')
    AND p1.name != 'None'
    AND p2.amenity != 'None'
AND p1.amenity != 'None'
GROUP by 1, 2
) AS T1
INNER JOIN (
    SELECT p1.name as no_ff_name, p1.way as no_ff_way, count(*) as no_fast_food
    FROM public.ph_point as p1 
    INNER JOIN public.ph_point as p2
    ON ST_dwithin(st_transform(p1.way,3123), st_transform(p2.way,3123),500)
    WHERE (p1.amenity = 'school' or p1.amenity = 'university')
    AND p1.name != 'None'
    AND p2.amenity != 'None'
    AND p1.amenity != 'None'
    AND p2.amenity != 'fast_food'
    GROUP by 1, 2
) AS T2
ON
T1.all_way = T2.no_ff_way AND
T1.all_name = T2.no_ff_name
""", conn)

df_combine.head(5)

,all_name,all_way,all_amenity,no_ff_name,no_ff_way,no_fast_food,fast_food
0,.,0101000020E6100000F1796A9A1CDC5E406CFF14D8AD57...,4,.,0101000020E6100000F1796A9A1CDC5E406CFF14D8AD57...,4,0
1,.,0101000020E6100000D7AD5406441D5F401E43119C44AA...,1,.,0101000020E6100000D7AD5406441D5F401E43119C44AA...,1,0
2,18th Avenue Elementary School,0101000020E610000073124A5F08445E4008C90226703B...,26,18th Avenue Elementary School,0101000020E610000073124A5F08445E4008C90226703B...,26,0
3,2,0101000020E6100000DEE522BE133D5E40F08AE07F2B69...,8,2,0101000020E6100000DEE522BE133D5E40F08AE07F2B69...,8,0
4,2nd Gear Technical Institute and Assessment Ce...,0101000020E6100000C4DCA45824405E40DBA09BA2C853...,5,2nd Gear Technical Institute and Assessment Ce...,0101000020E6100000C4DCA45824405E40DBA09BA2C853...,5,0


### Merge Schools in Metro Manila with their Densities

We have merge the Metro Manila schools with their densities and have also filtered to only pickup the Top 10 Percentile from the group as we want to focus on densed locations.

In [6]:
df_merge = df_school_mm.merge(df_combine, how='inner', left_on='way', right_on='all_way')
df_sch_sum = df_merge[['name', 'name_1', 'name_2', 'all_amenity', 'no_fast_food', 'fast_food', 'way']]
df_sch_sum.sort_values(by='all_amenity', ascending=False, inplace=True)
df_sch_sum = df_sch_sum[df_sch_sum.all_amenity >= df_sch_sum.all_amenity.quantile(.90)]
df_sch_sum.head(5)

,name,name_1,name_2,all_amenity,no_fast_food,fast_food,way
139,Informatics,Metropolitan Manila,Makati City,311,282,29,0101000020E6100000A051BAF42F415E4089EC832C0B1E...
141,Spurgeon School Foundation,Metropolitan Manila,Makati City,291,260,31,0101000020E61000000BB8E7F953415E409D256D4F351F...
142,Playworks School,Metropolitan Manila,Makati City,276,254,22,0101000020E61000002A96B61380415E40ED6C7EB2181F...
285,Review School of Accountants (ReSA),Metropolitan Manila,Manila,264,176,88,0101000020E61000004219F4EF443F5E404E603AADDB34...
284,Prime Review Center,Metropolitan Manila,Manila,264,176,88,0101000020E6100000AEBB79AA433F5E40F5F75278D034...


### Calculate Distance from AIM

As a final step, here we took the Top 10 density schools and calculated the distance from AIM as part of the key drivers for our decision with the expansion. We took the Lat and Lon coordinates of AIM in google maps and have converted it to the compatible SRID before computing the distance. 

In [12]:
print('Fetching Distance From AIM')
lat, lon = '14.552572634808579', '121.01870276413047' #AIM Lat, Lon 
ways = tuple(df_sch_sum.way.to_list())

df_aim_dist = pd.read_sql_query(
                  f"""
                  SELECT 
                      way,
                      ST_Distance(
                      ST_Transform(ST_SetSRID(ST_MakePoint({lon}, {lat}), 4326),3123),
                      ST_Transform(way, 3123))
                  FROM 
                      public.ph_point as p 
                  WHERE
                      way IN {ways}
                  """, 
             conn)
df_aim_dist.head(5)

Fetching Distance From AIM


,way,st_distance
0,0101000020E6100000418177F2E9415E4021C610A562D8...,14427.626108
1,0101000020E6100000F10005CD9D405E40E56F8CAE1CDF...,12960.369222
2,0101000020E6100000CBDCD7DCAC415E401465CAE259D9...,14191.194555
3,0101000020E610000047D38F3CC6405E40C1278C1CD81C...,824.495647
4,0101000020E6100000F0517FBDC23F5E40575BB1BFEC0E...,3544.760175


### List of Top Density Schools for McDollibee Expansion

There were 71 top locations for our consideration on the expansion

In [13]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
df_fin = df_aim_dist.merge(df_sch_sum, left_on='way', right_on='way')
df_fin = df_fin[['name', 'name_2', 'st_distance', 'all_amenity', 'no_fast_food', 'fast_food']]
df_fin.columns = ['School Name', 'City', 'Distance From AIM', 'Amenity Count', 'Amenity No Fast Food', 'Fast Food']
df_fin.sort_values(by='School Name', ascending=True)

,School Name,City,Distance From AIM,Amenity Count,Amenity No Fast Food,Fast Food
61,A1 Driving,Marikina,11849.047763,89,77,12
42,ACES Review Center,Manila,6765.900301,231,159,72
69,Asian Institute of Computer Studies,Marikina,12451.818700,195,181,14
66,Ateneo School of Government,Quezon City,11705.789678,163,157,6
50,BGPOP,Manila,6983.004617,171,113,58
4,Baclaran Elementary School Unit Iii,Parañaque,3544.760175,87,69,18
47,Besavilla Review Center,Manila,6692.203971,244,166,78
30,Brgy. 310 Day Care Center,Manila,6798.608551,227,153,74
52,CPAR,Manila,6761.594156,102,85,17
39,CRC ACE Review Center,Manila,6691.230373,243,165,78


### Top 10 Ordered by Distance from AIM Nearest to Farthest

In [14]:
display(df_fin.sort_values(by='Distance From AIM', ascending=True).head(10))

,School Name,City,Distance From AIM,Amenity Count,Amenity No Fast Food,Fast Food
7,Microcadd Institute Makati,Makati City,446.351110,144,133,11
9,Informatics,Makati City,675.802329,311,282,29
3,San Isidro National High School (Annex),Makati City,824.495647,141,124,17
10,Spurgeon School Foundation,Makati City,953.164800,291,260,31
11,iAcademy,Makati City,972.548282,237,215,22
12,Playworks School,Makati City,1037.533420,276,254,22
6,The Bridge,Makati City,1038.824910,163,143,20
8,Centro Escolar University,Makati City,1104.332900,166,146,20
19,NATIONAL MARITIME POLYTECHNIC Makati (NMP),Makati City,1350.126584,256,234,22
18,Little Gym,Makati City,2735.311929,194,169,25


### Top 10 Ordered by Amenity Count Highest to Lowest

In [15]:
display(df_fin.sort_values(by='Amenity Count', ascending=False).head(10))

,School Name,City,Distance From AIM,Amenity Count,Amenity No Fast Food,Fast Food
9,Informatics,Makati City,675.802329,311,282,29
10,Spurgeon School Foundation,Makati City,953.164800,291,260,31
12,Playworks School,Makati City,1037.533420,276,254,22
37,Prime Review Center,Manila,6473.457508,264,176,88
38,Review School of Accountants (ReSA),Manila,6477.443013,264,176,88
31,Review Center For Allied Professions (RCAP),Manila,6484.486819,261,175,86
29,Royal Philippines Educational Specialist,Manila,6543.805835,257,169,88
19,NATIONAL MARITIME POLYTECHNIC Makati (NMP),Makati City,1350.126584,256,234,22
34,SMART EDGE,Manila,6452.779343,256,168,88
24,STI,Manila,6474.882219,254,168,86


### Top 10 Ordered by Least Number of Competitors

In [16]:
display(df_fin.sort_values(by='Fast Food', ascending=True).head(10))

,School Name,City,Distance From AIM,Amenity Count,Amenity No Fast Food,Fast Food
66,Ateneo School of Government,Quezon City,11705.789678,163,157,6
65,Center For Culinary Arts,Quezon City,10925.496111,114,108,6
2,Optimal Academy,Muntinlupa,14191.194555,153,146,7
63,Headway School for Giftedness,Quezon City,11057.530594,101,94,7
57,Center for Asian Culinary Studies,San Juan,5341.852575,87,80,7
0,cambridge prep,Muntinlupa,14427.626108,130,121,9
68,MSA Academic Advancement Inst.,Quezon City,11267.714819,171,162,9
67,Mater Christi School,Quezon City,11203.564040,156,146,10
59,"Starland International School, Inc. (Cubao)",Quezon City,8282.743764,87,77,10
56,The Raya School,Quezon City,9052.241795,166,156,10


### Recommendation

To summarize what we did the data frame contains the Top 10 Percentile of Density based on Schools (Capture our Market) within Metro Manila to ensure we are within a reasonable distance from a logistic point of view. We added the distance from AIM to help with the decision-making. We have added columns for the density of amenities within the area, the count without fast food, and the count of fast food within the defined units of 500 which is subject to change depending on further preference. 

Given the data frame, we have we can decide within the vicinity of the school where we can potentially expand and open our next branch given the requirements. 

Of course, this is still subject to improvement as there are other variables we can pick up to further improve it:
1. Population Count within the area if any data is available specially # of students within the schools would matter.
2. Check the type of amenities if it's a type that will generate a lot of traffic since some amenities may not have many people even if there's a lot in an area.
3. Proximity where the branch would be located between schools since there might be multiple schools that can be captured even if other schools are small it can add up.
4. How exponential the cost will be further it will be since even if it's within Metro Manila if it's located like in Valenzuela its quite far still from AIM and we can say Laguna is still more accessible considering traffic.
5. Actual location of the school since we might be measuring midpoint and some schools might have a bigger campus and we may need to offset some for the radius.

To finalize I would get the most densed areas with high population of students ideally universities as they are the ones who can afford the meal, though radius is subjective this one we can still change. Less competition with fast food, additional improvement maybe consider restaurants however in this algo i don't have resto but its just additional filter then distance from AIM since the distance is reasonable for all since we are within Metro Manila. 



In [17]:
print('test')

test
